# Finetuning

This module contains some functions useful to finetune models.

In [ ]:
#| default_exp finetune

## Implementation

In [ ]:
#| export
from peft import PeftModel

/home/alex4321/anaconda3/envs/longdocchat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# These imports are only used in test
from llama_wrapper import import_llama
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch
import os

In [ ]:
#| export
def lora_model_zeros_and_scales_to_half(
        model: PeftModel # Original model
    ) -> PeftModel: # Converted model
    """
    Convert zeros and scales for PeftModel to half-precision
    """
    for _, m in model.named_modules():
        if "Autograd4bitQuantLinear" in str(type(m)) or "Linear4bitLt" in str(type(m)):
            if hasattr(m, "is_v1_model") and m.is_v1_model:
                m.zeros = m.zeros.half()
            m.scales = m.scales.half()
    return model

## Testing

To make a *very* simple test here I will just try to:

- see the perplexity of pretrained Vicuna model for the sample text file
- train the LoRA adapters on top of Vicuna
- see the perplexity of such a model

In [ ]:
_, train_data, load_llama_model_4bit_low_ram, _, model_to_half, _, apply_gradient_checkpointing, _, AMPWrapper = import_llama(
    use_flash_attention=False,
    use_xformers=False,
    autograd_4bit_cuda=False,
    autograd_4bit_triton=True
)

Using Triton implementation.


### Pretrained state

In [ ]:
if not os.path.exists("../vicuna-13b-GPTQ-4bit-128g"):
    !git clone "https://huggingface.co/anon8231489123/vicuna-13b-GPTQ-4bit-128g"
    !mv "vicuna-13b-GPTQ-4bit-128g" ..

In [ ]:
model_pretrained, tokenizer = load_llama_model_4bit_low_ram(
    config_path="../vicuna-13b-GPTQ-4bit-128g/",
    model_path="../vicuna-13b-GPTQ-4bit-128g/vicuna-13b-4bit-128g.safetensors",
    groupsize=128,
    is_v1_model=False,
)
tokenizer.pad_token_id = 0

Loading Model ...


The safetensors archive passed at ../vicuna-13b-GPTQ-4bit-128g/vicuna-13b-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.


Loaded the model in 3.27 seconds.


In [ ]:
dataset = train_data.TrainTxt(
    dataset="01_alpaca_text.txt",
    val_set_size=0,
    tokenizer=tokenizer,
    cutoff_len=256,
)
dataset.prepare_data(thd=-1, use_eos_token=1)

Train Data: 0.00% outliers


In [ ]:
model_to_half(model_pretrained)
model_pretrained_wrapper = AMPWrapper(model_pretrained)
model_pretrained_wrapper.apply_forward()

Converted as Half.


In [ ]:
def _test_model(model, data):
    probabilities = []
    with torch.no_grad():
        for sample in data:
            input_ids = torch.LongTensor([sample["input_ids"]]).cuda()
            response = model.forward(input_ids, return_dict=True)
            logits = response['logits'][0]
            probas = torch.nn.functional.softmax(logits, dim=-1)
            proba = probas.max(dim=-1).values.mean().item()
            probabilities.append(proba)
    average_proba = sum(probabilities) / len(probabilities)
    return average_proba

In [ ]:
model_pretrained.eval()
_test_model(model_pretrained, dataset.train_data)

0.703210663377193

In [ ]:
model_pretrained.cpu()
torch.cuda.empty_cache()

### Finetune

In [ ]:
model, tokenizer = load_llama_model_4bit_low_ram(
    config_path="../vicuna-13b-GPTQ-4bit-128g/",
    model_path="../vicuna-13b-GPTQ-4bit-128g/vicuna-13b-4bit-128g.safetensors",
    groupsize=128,
    is_v1_model=False,
)
tokenizer.pad_token_id = 0

Loading Model ...


The safetensors archive passed at ../vicuna-13b-GPTQ-4bit-128g/vicuna-13b-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.


Loaded the model in 2.38 seconds.


In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
)
lora_model = get_peft_model(model, lora_config)
lora_model = lora_model_zeros_and_scales_to_half(lora_model)

In [ ]:
apply_gradient_checkpointing(lora_model, checkpoint_ratio=1);

Forward Patch Applied For Block 0
Forward Patch Applied For Block 1
Forward Patch Applied For Block 2
Forward Patch Applied For Block 3
Forward Patch Applied For Block 4
Forward Patch Applied For Block 5
Forward Patch Applied For Block 6
Forward Patch Applied For Block 7
Forward Patch Applied For Block 8
Forward Patch Applied For Block 9
Forward Patch Applied For Block 10
Forward Patch Applied For Block 11
Forward Patch Applied For Block 12
Forward Patch Applied For Block 13
Forward Patch Applied For Block 14
Forward Patch Applied For Block 15
Forward Patch Applied For Block 16
Forward Patch Applied For Block 17
Forward Patch Applied For Block 18
Forward Patch Applied For Block 19
Forward Patch Applied For Block 20
Forward Patch Applied For Block 21
Forward Patch Applied For Block 22
Forward Patch Applied For Block 23
Forward Patch Applied For Block 24
Forward Patch Applied For Block 25
Forward Patch Applied For Block 26
Forward Patch Applied For Block 27
Forward Patch Applied For Bloc

In [ ]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    warmup_steps=5,
    optim="adamw_torch",
    num_train_epochs=10,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=20,
    evaluation_strategy="no",
    save_strategy="steps",
    eval_steps=None,
    save_steps=50,
    output_dir="lora-output-directory",
    save_total_limit=3,
    load_best_model_at_end=False,
    ddp_find_unused_parameters=False,
    report_to="none",
)

In [ ]:
trainer = Trainer(
    lora_model,
    train_dataset=dataset.train_data,
    eval_dataset=dataset.val_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
lora_model.config.use_cache = False

In [ ]:
trainer.train()

{'loss': 3.5212, 'learning_rate': 0.0002930973451327433, 'epoch': 0.35}


{'loss': 2.2487, 'learning_rate': 0.0002824778761061947, 'epoch': 0.7}


{'loss': 2.211, 'learning_rate': 0.00027185840707964596, 'epoch': 1.05}


{'loss': 2.0809, 'learning_rate': 0.00026123893805309734, 'epoch': 1.4}


{'loss': 1.7463, 'learning_rate': 0.00025061946902654866, 'epoch': 1.75}


...

{'loss': 1.3084, 'learning_rate': 2.0176991150442476e-05, 'epoch': 9.47}


{'loss': 1.5564, 'learning_rate': 9.557522123893805e-06, 'epoch': 9.82}


{'train_runtime': 1549.3327, 'train_samples_per_second': 0.368, 'train_steps_per_second': 0.368, 'train_loss': 1.52357904032657, 'epoch': 10.0}


100%|██████████| 570/570 [25:49<00:00,  2.72s/it]


TrainOutput(global_step=570, training_loss=1.52357904032657, metrics={'train_runtime': 1549.3327, 'train_samples_per_second': 0.368, 'train_steps_per_second': 0.368, 'train_loss': 1.52357904032657, 'epoch': 10.0})

### Tuned model

In [ ]:
lora_model.eval()
_test_model(lora_model, dataset.train_data)

/home/alex4321/anaconda3/envs/longdocchat/lib/python3.11/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


0.8225835499010588